In [1]:
features = ['normal', 'dry', 'oily', 'combination', 'acne', 'sensitive', 'fine lines', 'wrinkles', 'redness',
            'dull', 'pore', 'pigmentation', 'blackheads', 'whiteheads', 'blemishes', 'dark circles', 'eye bags', 'dark spots','dark', 'light', 'mid-dark', 'mid-light']

In [2]:
def generate_vector(skin_type, concerns,skin_tone):
    vector = [0] * len(features)

    # Set skin type
    if skin_type in features[:5]:
        vector[features.index(skin_type)] = 1

    # Set concerns
    for concern in concerns:
        if concern in features[4:18]:
            vector[features.index(concern)] = 1

    if skin_tone in features[18:]:
        vector[features.index(skin_tone)] = 1

    return vector




# Example usage
skin_type = 'oily'
concerns = ['Level_2', 'redness']

acne_l = ''

for i in concerns:
    if i in ['Level_1','Level_2','Level_3']:
        acne_l = i
        concerns.append('acne')
        break

total_concerns = concerns + ['general care','sun protection']
skin_tone = 'dark'
x = generate_vector(skin_type, concerns,skin_tone)
print(x)


[0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]


In [3]:

from IPython.utils.tempdir import TemporaryWorkingDirectory
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

df2 = pd.read_csv('final_with_skin_tone.csv')
#makeup = pd.read_csv('makeup_final.csv')
entries = len(df2)
LABELS = list(df2.label.unique())
# features

features = ['normal', 'dry', 'oily', 'combination', 'acne', 'sensitive', 'fine lines', 'wrinkles', 'redness',
            'dull', 'pore', 'pigmentation', 'blackheads', 'whiteheads', 'blemishes', 'dark circles', 'eye bags', 'dark spots','dark', 'light', 'mid-dark', 'mid-light']



# utility functions

def search_concern(target, i):
    if target in df2.iloc[i]['concern']:
        return True
    return False


def name2index(name):
    return df2[df2["name"] == name].index.tolist()[0]


def index2prod(index):
    return df2.iloc[index]


def wrap(info_arr):
    result = {}
#     print(info_arr)
    result['brand'] = info_arr[0]
    result['name'] = info_arr[1]
    result['price'] = info_arr[2]
    result['url'] = info_arr[3]
    result['img'] = info_arr[4]
    result['skin type'] = info_arr[5]
    result['concern'] = str(info_arr[6]).split(',')
    result['skin tone'] = info_arr[7]
    return result

def wrap_makeup(info_arr):
    result = {}
#     print(info_arr)
    result['brand'] = info_arr[0]
    result['name'] = info_arr[1]
    result['price'] = info_arr[2]
    result['url'] = info_arr[3]
    result['img'] = info_arr[4]
    result['skin type'] = info_arr[5]

    return result

one_hot_encodings = np.zeros([entries, len(features)])


#skin types first
for i in range(entries):
    for j in range(5):
        target = features[j]
        sk_type = df2.iloc[i]['skin type']
        if sk_type == 'all':
            one_hot_encodings[i][0:5] = 1
        elif target == sk_type:
            one_hot_encodings[i][j] = 1

#other features
for i in range(entries):
    for j in range(4, len(features)):
        feature = features[j]
        if feature in df2.iloc[i]['concern']:
            one_hot_encodings[i][j] = 1

for i in range(entries):
    for j in range(18, len(features)):
        target = features[j]
        sk_type = df2.iloc[i]['skin tone']
        if sk_type == 'all':
            one_hot_encodings[i][18:22] = 1
        elif target == sk_type:
            one_hot_encodings[i][j] = 1

# recommend top 5 similar items from a category


def recs_cs(vector = None, name = None, label = None, count = 5):
    products = []
    if name:
        idx = name2index(name)
        fv = one_hot_encodings[idx]
    elif vector:
        fv = vector
        #print(fv)
    cs_values = cosine_similarity(np.array([fv, ]), one_hot_encodings)
    df2['cs'] = cs_values[0]

    if label:
        dff = df2[df2['label'] == label]
    else:
        dff = df2

    if name:
        dff = dff[dff['name'] != name]
    recommendations = dff.sort_values('cs', ascending=False).head(count)
    #   print(f"Top {count} matching {label} items")
    data = recommendations[['brand', 'name', 'price', 'url','img','skin type','concern','skin tone']].to_dict('split')['data']
    for element in data:
        products.append(wrap(element))
    return products

    # overall recommendation


def recs_essentials(vector = None, name = None):
#     print("ESSENTIALS:")
    response = {}
    for label in LABELS:
#         print(f"{label}:")
        if name:
            r = recs_cs(None, name, label)
        elif vector:
            r = recs_cs(vector, None, label)
        response[label] = r
    return response




In [4]:
x = recs_essentials(x, None)

In [5]:
for i,j in x.items():
    for k in j:
        if acne_l != '':
            k['concern'].append(acne_l)
        k['concern'] = [x for x in set(k['concern']) if x != '']
        if any(con in k['concern'] for con in total_concerns):
            print(i,k)

face-moisturisers {'brand': 'ustraa', 'name': 'men oily skin moisturising cream 100g', 'price': '₹ 224', 'url': 'https://www.myntra.com/face-moisturisers/ustraa/ustraa-men-oily-skin-moisturising-cream-100g/13220398/buy', 'img': 'https://assets.myntassets.com/h_720,q_90,w_540/v1/assets/images/13220398/2021/3/15/2395b329-d3a9-4b57-a8bf-2a59f51019781615809377964-Ustraa-Men-Oily-Skin-Moisturising-Cream-100g-620161580937701-1.jpg', 'skin type': 'oily', 'concern': ['Level_2', 'smoothening', 'excess oil', 'softening'], 'skin tone': 'dark'}
face-moisturisers {'brand': 'skinkraft', 'name': 'customized moisturizer for oily skin - 45 ml', 'price': '₹ 499', 'url': 'https://www.myntra.com/face-moisturisers/skinkraft/skinkraft-customized-moisturizer-for-oily-skin---45-ml/15345052/buy', 'img': 'https://assets.myntassets.com/h_720,q_90,w_540/v1/assets/images/15345052/2021/9/2/00a22bbe-f828-47c0-aed6-6a68b431fcfa1630588600147SKINKRAFT1.jpg', 'skin type': 'oily', 'concern': ['excess oil', 'Level_2', 'hy

In [6]:
print(acne_l)

Level_2
